# Deep Learning (RNN) Demo for Load Forecasting

### Step 1: Import all the packages needed

In [1]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl

#### Step 2.1: setting all global parameters -- sec 1 data parameters

In [2]:
time1 = time.time()
data_path = './input.csv'
total_days = 350
train_days = 280
test_days = 70
data_length = 0

#### Step 2.2: setting all global parameters -- sec 2 network configuration

In [3]:
num_epoches = 100
n_steps = 48 # input size
batch_size = 70*48 # days of a batch
feature_size = 1 # same time of a week
n_hidden = 5 # input size
num_layers = 2
n_output = 1

### Step 3:loading data

In [4]:
dataframe = pd.read_csv(data_path)
dat = np.array(dataframe)
date_list = dat[:,1]
dat = dat[:,2:]# drop the first two cols --- index and date
nrows,ncols = dat.shape
#print nrows,ncols
data = dat.reshape((1,nrows*ncols))
data_length = data.shape[1]

# construct training data
train_len = train_days*48
train_data = data[0,0:train_len]
train_data = train_data.reshape([1,train_len])

# construct testing data
## test size = input_size + test days size. since, the output should be 
## from first test sample to last. prefix is input-size data
test_len = test_days*48
test_data = np.zeros(test_len+n_steps)
test_data[n_steps:] = data[0,train_len:train_len+test_len]
test_data[0:n_steps] = data[0,train_len-n_steps:train_len]
test_data = test_data.reshape([1,test_len+n_steps])
print "train data shape: {}, test data shape: {}".format(train_data.shape,test_data.shape)

train data shape: (1, 13440), test data shape: (1, 3408)


### Step 4: define data generating function code. 
which generate a batch of batch-size large sequence data. the data is feature_size dims width and is a time series of float32 of steps steps. inputs and outputs are:

inputs:
----n_batch: number of samples in a batch
----steps: the sequence length of a sample data
----feature_size: dimensions of a single time step data frame

outputs:
----X inputs, shape(n_batch,steps,feature_size)
----Y outputs should be, shape(n_batch,)

In [5]:
def train_data_gen(steps = 48, n_batch = 48):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    #for each n, compute X and correct y values
    for n in range(n_batch):
        # randomly pick a sample's y, between acceptable range
        index = np.random.randint(steps,train_len)
        # update y
        Y[n] = train_data[0,index]
        # update X from index-steps to index-1
        X[n,:,:] = train_data[0,index-steps:index].T.reshape(steps,1)
    return (X,Y)

In [6]:
def test_data_gen(steps = 7*48, n_batch = 70*48):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    #for each n, compute X and correct y values
    for n in range(n_batch):
        # randomly pick a sample's y, between acceptable range
        index = steps
        # update y
        Y[n] = test_data[0,steps+n]
        # update X from index-steps to index-1
        X[n,:,:] = test_data[0,n:n+steps].T.reshape(steps,1)
    return (X,Y)

### Step 5: construct RNN model

In [7]:
# create placeholder for x and y
x = tf.placeholder("float",[None,n_steps,feature_size])
istate = tf.placeholder("float",[None,num_layers*2*n_hidden])
y = tf.placeholder("float",[None,n_output])


# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([feature_size, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [8]:
def RNN(_X, _istate, _weights, _biases):

    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, feature_size]) # (n_steps*batch_size, n_input)
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    stacked_lstm_cell = rnn_cell.MultiRNNCell([lstm_cell]*num_layers)
    
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(stacked_lstm_cell, _X, initial_state=_istate)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [9]:
pred = RNN(x, istate, weights, biases)

#cost function 
cost = tf.reduce_mean(tf.pow(pred-y,2)) # cost function of this batch of data
#cost2 = tf.abs(cost1)
#compute parameter updates
#train_op = tf.train.GradientDescentOptimizer(0.008).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost)

### Step 6: generate validation data

In [10]:
x_val,y_val = test_data_gen(n_steps,batch_size)
print "test data: x_val shape - {}; y_val shape - {}".format(x_val.shape,y_val.shape)

test data: x_val shape - (3360, 48, 1); y_val shape - (3360, 1)


### Step 7: run rnn network

In [11]:
### Execute
# Initializing the variables
init = tf.initialize_all_variables()
outp = []
with tf.Session() as sess:
    # Create a summary to monitor cost function
    tf.scalar_summary("loss", cost)
    #tf.scalar_summary("loss2",cost2)
    # Merge all summaries to a single operator
    merged_summary_op = tf.merge_all_summaries()

    # tensorboard info.# Set logs writer into folder /tmp/tensorflow_logs
    summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def=sess.graph_def)
    
    #initialize all variables in the model
    sess.run(init)
    for k in range(num_epoches):
        #Generate Data for each epoch
        #What this does is it creates a list of of elements of length seq_len, each of size [batch_size,input_size]
        #this is required to feed data into rnn.rnn
        X,Y = train_data_gen(n_steps,batch_size)
        X = X.reshape(batch_size,n_steps,feature_size)
        
        
        #Create the dictionary of inputs to feed into sess.run
        sess.run(optimizer,feed_dict={x:X,y:Y,istate:np.zeros((batch_size,num_layers*2*n_hidden))})   
        #perform an update on the parameters
        
        loss1 = sess.run(cost, feed_dict = {x:X,y:Y,istate:np.zeros((batch_size,num_layers*2*n_hidden))} )
        loss2 = sess.run(cost, feed_dict = {x:x_val,y:y_val,istate:np.zeros((batch_size,num_layers*2*n_hidden))} )            #compute the cost on the validation set
        output_tmp = sess.run(pred,feed_dict = {x:X,y:Y,istate:np.zeros((batch_size,num_layers*2*n_hidden))} )
        outp_train = output_tmp
        output_tmp = sess.run(pred,feed_dict = {x:x_val,y:y_val,istate:np.zeros((batch_size,num_layers*2*n_hidden))} )
        outp_test = output_tmp
        
        if k == num_epoches-1:
            outp_train = np.array(outp_train)
            xxx = np.arange(0,test_len)
            pl.plot(xxx,outp_train,color = "red")
            pl.plot(xxx,Y)
            pl.grid()
            pl.legend()
            pl.show()
            R = np.corrcoef(outp_train.T,Y.T)
            RR = R**2
            print "R: {}, R-square: {}".format(R,RR)
            
        # Write logs at every iteration
        summary_str = sess.run(merged_summary_op, feed_dict={x:x_val,y:y_val,istate:np.zeros((batch_size,num_layers*2*n_hidden))} )
        summary_writer.add_summary(summary_str, k)
        print "Iter " + str(k) + ", Minibatch Loss ---- Train = " + "{:.6f}".format(loss1) + "; Test = " + "{:.6f}".format(loss2)
    #print "haha{}".format(outp)

Iter 0, Minibatch Loss ---- Train = 0.093655; Test = 0.177809
Iter 1, Minibatch Loss ---- Train = 0.081665; Test = 0.174156
Iter 2, Minibatch Loss ---- Train = 0.076619; Test = 0.172438
Iter 3, Minibatch Loss ---- Train = 0.074930; Test = 0.172424
Iter 4, Minibatch Loss ---- Train = 0.075649; Test = 0.171902
Iter 5, Minibatch Loss ---- Train = 0.068811; Test = 0.167265
Iter 6, Minibatch Loss ---- Train = 0.066549; Test = 0.165730
Iter 7, Minibatch Loss ---- Train = 0.069917; Test = 0.159447
Iter 8, Minibatch Loss ---- Train = 0.060479; Test = 0.160214
Iter 9, Minibatch Loss ---- Train = 0.060076; Test = 0.153681
Iter 10, Minibatch Loss ---- Train = 0.054533; Test = 0.153607
Iter 11, Minibatch Loss ---- Train = 0.052774; Test = 0.148411
Iter 12, Minibatch Loss ---- Train = 0.048966; Test = 0.147337
Iter 13, Minibatch Loss ---- Train = 0.047992; Test = 0.143373
Iter 14, Minibatch Loss ---- Train = 0.046384; Test = 0.143013
Iter 15, Minibatch Loss ---- Train = 0.044880; Test = 0.139634
It

/usr/lib/pymodules/python2.7/matplotlib/axes.py:4747: UserWarning: No labeled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labeled objects found. "


R: [[ 1.          0.66656463]
 [ 0.66656463  1.        ]], R-square: [[ 1.          0.44430841]
 [ 0.44430841  1.        ]]
Iter 99, Minibatch Loss ---- Train = 0.022988; Test = 0.090434


### Step 8: Evaluation

In [12]:
out = np.array(outp_test)
out.shape

(3360, 1)

In [13]:
out.shape

(3360, 1)

In [14]:
y_val.dtype = float
y_val.shape

(3360, 1)

In [15]:
R = np.corrcoef(out.T,y_val.T)
RR = R**2

In [16]:
# final R
R

array([[ 1.        ,  0.58340399],
       [ 0.58340399,  1.        ]])

In [17]:
# final R-square
RR

array([[ 1.        ,  0.34036022],
       [ 0.34036022,  1.        ]])

In [18]:
xxx = np.arange(0,test_len)
pl.plot(xxx,out,color = "red")
pl.plot(xxx,y_val)
pl.grid()
pl.legend()
pl.show()

In [19]:
# final MSE
sq = pow(out-y_val,2)
np.mean(sq)

0.09043369367934595

In [20]:
# run time
time2 = time.time()
print time2-time1

94.7019429207


In [21]:
help(pl.subplot)

Help on function subplot in module matplotlib.pyplot:

subplot(*args, **kwargs)
    Return a subplot axes positioned by the given grid definition.
    
    Typical call signature::
    
      subplot(nrows, ncols, plot_number)
    
    Where *nrows* and *ncols* are used to notionally split the figure
    into ``nrows * ncols`` sub-axes, and *plot_number* is used to identify
    the particular subplot that this function is to create within the notional
    grid. *plot_number* starts at 1, increments across rows first and has a
    maximum of ``nrows * ncols``.
    
    In the case when *nrows*, *ncols* and *plot_number* are all less than 10,
    a convenience exists, such that the a 3 digit number can be given instead,
    where the hundreds represent *nrows*, the tens represent *ncols* and the
    units represent *plot_number*. For instance::
    
      subplot(211)
    
    produces a subaxes in a figure which represents the top plot (i.e. the
    first) in a 2 row by 1 column notiona

In [22]:
Y

array([[ 0.14 ],
       [ 0.111],
       [ 0.072],
       ..., 
       [ 0.113],
       [ 0.446],
       [ 0.113]])